In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
from multiprocessing import Pool
from itertools import chain
import numpy as np

In [2]:
def get_links(path):
    # Now we get html code from page
    soup = BeautifulSoup(urlopen(path), 'html.parser')
    
    
    # We extract the href corresponding to each car
    extracted_href = soup.findAll("a", {"href": re.compile("//www.leboncoin.fr/voitures/*")})
    s = str(extracted_href)
    
    # To have the titles corresponding to the links, we extract bot the links and titles for /voitures/
    # And store only the titles, because they are clean.
    titles = re.findall(r'title="(\S.*)"', s)
    titles = titles[:len(titles)-1]
    
    # title gathered one value more than it should have, it will always be the last one because it is in the page's footer.
    
    
    # Now we extract the links corresponding to cars ads. 
    links = re.findall(r'www.leboncoin.fr/voitures/\d+\.\w*.\w*.\w*', s)
    return links

In [3]:
# Prendre les derniers chiffres pour les kilomètres, en général c'est 5 chiffres...
def check_number_digits_km(km):
    if(km == 'NA'):
        return 'NA'
    else:
        s = str(km)
        if(len(s)> 5):
             s= s[-5:]
        return int(s)

In [4]:
def extract_info_from_link(link):
    # Now we get html code from page
    soup2 = BeautifulSoup(urlopen("https://" + link), 'html.parser')
    
    # We extract title of ad:
    extracted_title = soup2.find("h1", {"class":"no-border"}).text.strip()
    
    
    # We extract the price
    extracted_price = soup2.findAll("h2", {"class": re.compile("item_price clearfix")})
    price = re.findall(r'content="(.*?)"',str(extracted_price))[0] # price contient [prix, Currency]
    
    # We extract the year
    extracted_year = soup2.findAll("span", {"itemprop": re.compile("releaseDate")})
    year = extracted_year[0].text.strip()
    
    # Now we want to extract phone number, model, and kilometers of the car in the description:
    # We first extract the description and convert it to a string:
    extracted_description = soup2.findAll("div", {"class": "line properties_description"})
    extracted_description = str(extracted_description[0])
    
    # Extraction of the kilometers of the car:
    km = re.findall(r'(\d+).?(\d*)\s*(km|kms|KM|KMS|Km|Kms)', str(extracted_description))
    # This can find a lot of different values for the kilometers, we admit that the first one is the one we want.
    # But we check as well if we extracted at least one value for kilometers. If not we set it to NA
    if(len(km)>0):
        elements_of_km = ""
        for item in km[0][:len(km[0])-1]:
            elements_of_km = elements_of_km + item
        km = elements_of_km
    else:
        km="NA"
    km = check_number_digits_km(km)
    
    # Extraction of the car's model, it can be between 3 values, Zen, Intens and Life.
    model = re.findall(r'(zen|intens|life|Zen|Intens|Life)', str(extracted_description))
    if(len(model)>0):
        model = model[0]
    else:
        model = "NA"
        
    # Extraction of the seller's phone number:
    phone_number = re.findall(r'0[1-9]\d{8}', str(extracted_description))
    if(len(phone_number)>0):
        phone_number = phone_number[0]
    else:
        phone_number = "NA"
        
    # Check if seller is a pro or not:
    extracted_pro = soup2.findAll("span", {"class": "ispro"})
    if len(extracted_pro) > 0:
        ispro = True
    else:
        ispro = False
    
    # Return description, it could be useful:
    description = re.sub(r'<[^>]*>', " ", str(extracted_description))
    
    description.upper()
    
    return extracted_title, price, year, km, model, phone_number, ispro, description

In [5]:
def get_zoe_links_for_region(region):
    print("start")
    root_link = "https://www.leboncoin.fr/annonces/offres/"
    car = "renault%20zo%E9"
    page = 1
    path = root_link + region + "/?th=" + str(page) + "&q=" + car
    links = []
    links.append(get_links(path))
    continuing = True
    while continuing == True :
        page = page + 1
        path = root_link + region + "/?th=" + str(page) + "&q=" + car
        new_links = get_links(path)
        if len(new_links) > 0:
            links.append(new_links)
        else:
            continuing = False
    print("end")
    return list(chain.from_iterable(links))


In [6]:
regions = ["ile_de_france", "aquitaine", "provence_alpes_cote_d_azur"]

links = list(map(lambda x: get_zoe_links_for_region(x), regions))
links = list(chain.from_iterable(links))
#links

start
end
start
end
start
end


In [7]:
pool = Pool(processes=8)
df_zoe = pool.map(extract_info_from_link, links)
df_zoe = pd.DataFrame(df_zoe, columns=["Title","Price","Year","km","Model","Phone","IsPro","Description"])

Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cell

In [8]:
df_zoe

,Title,Price,Year,km,Model,Phone,IsPro,Description
0,Zoe 40 bleu foudre my2017,15900,2017,NA,NA,NA,False,\n Description : \n Vends ZOE 40 BLEU FOUDRE ...
1,RENAULT ZOE INTENS Charge rapide Année 2015 Ca...,10990,2015,NA,NA,NA,True,\n Description : \n RENAULT ZOE INTENS TYPE 2...
2,Renault Zoe Life charge rapide,7900,2014,NA,Life,0134301285,True,\n Description : \n Renault Zoe Life charge r...
3,RENAULT ZOE LIFE GPS Clim auto Radar Très bel ...,8990,2013,NA,NA,NA,True,\n Description : \n RENAULT ZOE LIFE - DU 22/...
4,Renault Zoé Life 2014,9700,2014,160,Life,NA,False,"\n Description : \n Renault Zoé Life, modèle ..."
5,Nouvelle Renault Zoé intens blanche 400 km,16800,2017,4000,NA,NA,False,\n Description : \n Je suis collaborateur Ren...
6,Renault Zoe LIFE,13990,2017,NA,NA,0160657097,True,"\n Description : \n Renault Zoe LIFE berline,..."
7,Renault Zoe LIFE,10690,2015,NA,NA,0178455073,True,"\n Description : \n Renault Zoe LIFE berline,..."
8,Renault Zoe LIFE CHARGE RAPIDE,10300,2015,NA,NA,0175448151,True,\n Description : \n Renault Zoe LIFE CHARGE R...
9,Renault Zoé Type 2 Charge Rapide 2016,8900,2016,NA,Life,NA,False,\n Description : \n Renault Zoe Life Type 2 b...


In [9]:
def check_title_model(title):
    model = re.findall(r'(zen|intens|life|Zen|Intens|Life|ZEN|INTENS|LIFE)', title)
    if(len(model)>0):
        return model[0]
    else:
        return "NA"

df_zoe['Model'].loc[df_zoe['Model'] == "NA"] = list(map(lambda x: check_title_model(x), df_zoe['Title'].loc[df_zoe['Model'] == "NA"]))
df_zoe['Model'] = list(map(lambda x: x.upper(), df_zoe['Model']))
df_zoe['Description'] = list(map(lambda x: x.upper(), df_zoe['Description']))
df_zoe.head(5)

/Users/remi/Desktop/Cours/StarterKitBD/StarterKit_venv3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Title,Price,Year,km,Model,Phone,IsPro,Description
0,Zoe 40 bleu foudre my2017,15900,2017,NA,NA,NA,False,\n DESCRIPTION : \n VENDS ZOE 40 BLEU FOUDRE ...
1,RENAULT ZOE INTENS Charge rapide Année 2015 Ca...,10990,2015,NA,INTENS,NA,True,\n DESCRIPTION : \n RENAULT ZOE INTENS TYPE 2...
2,Renault Zoe Life charge rapide,7900,2014,NA,LIFE,0134301285,True,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...
3,RENAULT ZOE LIFE GPS Clim auto Radar Très bel ...,8990,2013,NA,LIFE,NA,True,\n DESCRIPTION : \n RENAULT ZOE LIFE - DU 22/...
4,Renault Zoé Life 2014,9700,2014,160,LIFE,NA,False,"\n DESCRIPTION : \n RENAULT ZOÉ LIFE, MODÈLE ..."


In [10]:
def extract_links_argus(year):
    path = "https://www.lacentrale.fr/cote-voitures-renault-zoe--" + str(year) + "-.html"
    soup3 = BeautifulSoup(urlopen(path), 'html.parser')
    
    # We extract values and labels for the ones that are in bold in the html code 
    extracted_href = soup3.findAll("a", {"href": re.compile("cote-auto-renault-zoe-*")})
    links_argus = re.findall(r'cote-auto-renault-zoe-(.*).html', str(extracted_href))
    return links_argus, list(np.repeat(year, len(links_argus)))
#links_argus = extract_links_argus(path_argus)

In [11]:
def get_cote(path):
    soup4 = BeautifulSoup(urlopen(path), 'html.parser')
    # We extract values and labels for the ones that are in bold in the html code 
    extracted_cote = soup4.findAll("span", {"class": "jsRefinedQuot"})
    extracted_model = soup4.findAll("span", {"class": "sizeC clear txtGrey7C sizeC"})

    return extracted_cote[0].text.strip(), extracted_model[0].text.strip()
#get_cote(path_cote)

In [14]:
years = [2013, 2014, 2015, 2016, 2017]
links_argus = list(map(lambda x: extract_links_argus(x), years))


links_argus = list(chain.from_iterable(links_argus))
years = links_argus[1::2]
years = list(chain.from_iterable(years))
links_argus = links_argus[0::2]
links_argus = list(chain.from_iterable(links_argus))

links_argus = list(map(lambda x: "https://www.lacentrale.fr/cote-auto-renault-zoe-" + str(x) + ".html", links_argus))
links_argus

['https://www.lacentrale.fr/cote-auto-renault-zoe-intens+charge+rapide-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-intens+charge+rapide+type+2-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+type+2-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-zen+charge+rapide-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-zen+charge+rapide+type+2-2013.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-intens+charge+rapide-2014.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-intens+charge+rapide+type+2-2014.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide-2014.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-life+charge+rapide+type+2-2014.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-zen+charge+rapide-2014.html',
 'https://www.lacentrale.fr/cote-auto-renault-zoe-zen+charge+rapide+t

In [15]:
df_argus = pool.map(get_cote, links_argus)
df_argus = pd.DataFrame(df_argus, columns = ["Price", "Model"])
df_argus["Year"] = years
df_argus.head(5)

,Price,Model,Year
0,8 410,INTENS CHARGE RAPIDE,2013
1,9 944,INTENS CHARGE RAPIDE TYPE 2,2013
2,9 335,LIFE CHARGE RAPIDE,2013
3,9 286,LIFE CHARGE RAPIDE TYPE 2,2013
4,10 744,ZEN CHARGE RAPIDE,2013


In [16]:
# On créé pour argus une colonne modele en extrayant Zen, LIFE, on intens.
def get_first_word(sentence):
    split_sentence = sentence.split(" ")
    return split_sentence[0]
def delete_first_word(sentence):
    split_sentence = sentence.split(" ")
    if len(split_sentence) > 1:
        return ' '.join(split_sentence[1:])
    else:
        return sentence
df_argus["Gamme"] = list(map(lambda x: get_first_word(x), df_argus["Model"]))
df_argus["Model"] = list(map(lambda x: delete_first_word(x), df_argus['Model']))
df_argus.head(5)

,Price,Model,Year,Gamme
0,8 410,CHARGE RAPIDE,2013,INTENS
1,9 944,CHARGE RAPIDE TYPE 2,2013,INTENS
2,9 335,CHARGE RAPIDE,2013,LIFE
3,9 286,CHARGE RAPIDE TYPE 2,2013,LIFE
4,10 744,CHARGE RAPIDE,2013,ZEN


In [17]:
def check_cote(model, year, description):
    
    df_filtered = df_argus
    # On filtre sur l'année dans un premier temps
    df_filtered = df_filtered.loc[df_filtered['Year'] == int(year)].loc[df_filtered['Gamme'] == model]
    #print(df_filtered)
    # On filtre ensuite sur la gamme
    
    # Maintenant il faut vérifier si notre voiture est parmis l'un des modèles:
    count = 0
    for gamme in df_filtered['Model']:
        count = count +1
        #print(gamme)
        if gamme in description:
            #print(gamme)
            #print(count)
            return df_filtered['Price'].loc[df_filtered["Model"] == gamme].values[0].strip()
    return 'NA'

df_zoe['Argus'] = list(map(check_cote, df_zoe["Model"], df_zoe['Year'], df_zoe['Description']))
df_zoe.head(5)

,Title,Price,Year,km,Model,Phone,IsPro,Description,Argus
0,Zoe 40 bleu foudre my2017,15900,2017,NA,NA,NA,False,\n DESCRIPTION : \n VENDS ZOE 40 BLEU FOUDRE ...,NA
1,RENAULT ZOE INTENS Charge rapide Année 2015 Ca...,10990,2015,NA,INTENS,NA,True,\n DESCRIPTION : \n RENAULT ZOE INTENS TYPE 2...,11 835
2,Renault Zoe Life charge rapide,7900,2014,NA,LIFE,0134301285,True,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...,10 212
3,RENAULT ZOE LIFE GPS Clim auto Radar Très bel ...,8990,2013,NA,LIFE,NA,True,\n DESCRIPTION : \n RENAULT ZOE LIFE - DU 22/...,9 335
4,Renault Zoé Life 2014,9700,2014,160,LIFE,NA,False,"\n DESCRIPTION : \n RENAULT ZOÉ LIFE, MODÈLE ...",NA
